### Interaction with Jupyter Notebook remotely

The notebook does the following

- Connects to user jupyter session using the token
- Queries notebook content in the server
- Queries running kernels
- Executes code on jupyter kernel
- Downloads data 


### Create a new object to communicate with the server

In [18]:
from jupyterhub_client import  RemoteJupyterClient
import pprint
pp = pprint.PrettyPrinter(indent=4)

remote = RemoteJupyterClient('ubuntu','8a03715de83345558f8d8bde2bd5f3da','rstudio2.luoquant.com/julia')

### Lists down running kernels. At least one kernel should be available to execute the code

In [19]:
remote.kernels()

/home/ubuntu/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rstudio2.luoquant.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


""


In [25]:
# Launch a new terminal
terminal = remote._post_(svc = 'terminals', path = '', payload = {})

/home/ubuntu/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rstudio2.luoquant.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [26]:
terminal

{'name': '2'}

In [22]:
import requests
url =  remote._url_(svc='terminals', path = '')
print(url)
response = requests.post(url,headers=remote.headers, verify = remote.verify, json = {})

https://rstudio2.luoquant.com/julia/user/ubuntu/api/terminals


/home/ubuntu/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'rstudio2.luoquant.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [24]:
response.content

b'{"name": "1"}'

In [48]:
import ssl
import websocket
self = remote
terminal = 1
channel = "wss://" + self.url + "/user/" + self.user + "/terminals/websocket/" + str(terminal)
ws = websocket.WebSocket(sslopt ={"cert_reqs": ssl.CERT_NONE})
terminal = ws.connect(url = channel, header = self.headers)

In [53]:
terminal = ws.connect(url = channel, header = self.headers)

In [54]:
terminal

In [17]:
from IPython.core.display import display, HTML
display(HTML(response.content.decode('utf-8')))

### Read content of a notebook

In [ ]:
notebook = remote.content('projects/demo/jupyter-remote/Remote Jupyter Demo.ipynb')

In [ ]:
pp.pprint(notebook.loc[0,'source'])

In [ ]:
pp.pprint(notebook.loc[1,'source'])

### Execute Remote Code on the server

In [ ]:
response = remote.execute('12263c47-20c1-4311-b93a-bf79911e8042', notebook.loc[0,'source'])

In [ ]:
response = remote.execute('12263c47-20c1-4311-b93a-bf79911e8042', notebook.loc[1,'source'])

### Download Data from the server

In [ ]:
remote.content('projects/demo/jupyter-remote')

In [ ]:
data = remote.content('projects/demo/jupyter-remote/sp500.csv')

In [ ]:
??remote.content

In [ ]:
data

In [ ]:
response = remote.execute('12263c47-20c1-4311-b93a-bf79911e8042', notebook.loc[2,'source'])

In [ ]:
## Server Code
port = wq.port_upload_file('MYCDTEST','/mnt/ebs1/karora/projects/demo/datafiles/mydata.csv',
                                _global=False,pit_id=True,short_format=False)

In [ ]:
import pandas as pd
import json

def is_server_code(code):
    
    #print(type(code))
    if type(code) == list:
        if len(code) < 1:
            return False
        return code[0].startswith('## Server Code')
    
    if type(code) == str:
        return code.startsWith('## Server Code')
    
    return False

with open('RemoteJupyter.ipynb') as f:
    v = json.load(f)
    local_notebook = pd.DataFrame(v['cells'])
    local_notebook = local_notebook[local_notebook.cell_type == 'code']
    local_notebook = local_notebook[[is_server_code(x) for x in local_notebook.source]]


In [ ]:
local_notebook

In [ ]:
response = remote.execute('12263c47-20c1-4311-b93a-bf79911e8042', "".join(local_notebook.iloc[0].source))

In [ ]:
response.is_error()

In [ ]:
## Server Code
print("Fetching Data....")
r1 = wq.new_request().runFor('CIQMSCI_379947354').start('2022-01-31').to('2022-04-30').at('1me').a('TICKER').addInFlag()
data1 = wq.get_data(r1)
print("Fetching Data Done.")
df = data1.as_large_data_frame()
print("Exporting to File")
df.to_csv('acwi.csv')
print("File available for download")

In [ ]:
response = remote.execute('12263c47-20c1-4311-b93a-bf79911e8042', "".join(local_notebook.iloc[1].source))